In [1]:
%autosave 30
%load_ext autoreload
%autoreload 2

Autosaving every 30 seconds


In [2]:
cd ..

/home/huawei123/kwx1991442/code-classification


In [63]:
import logging
from typing import *

from torch import Generator
from torch.utils.data import random_split, DataLoader

from src.baseline.dataloader import UITestsDataset
from src.word2vec.dataloader import W2VClassificationCollator, TrainW2VModel
from src.params import PATH_TEST_UI, PATH_PARSED_CLASSIFUI

logger = logging.getLogger(__name__)

In [64]:
from src.main import parse_args

args = parse_args(True, ["--method", "word2vec", "--tokens-source", "origin"])

In [65]:
args

Namespace(batch_size=4, classifier='bayes', device='cuda', exp_name='TEST', learning_rate=5e-05, liblinear_params='-s 5', max_seq_len=512, method='word2vec', mode='eval', n_epochs=8, save_all=False, save_metrics=False, save_model=False, save_predictions=False, seed=42, tokens_source='origin', traintestsplit=0.7, verbose=False, w2v_concat_method='mean', w2v_epochs=50, w2v_min_count=5, w2v_vector_size=500, w2v_window=500)

In [67]:
if args.mode != 'eval':
    raise NotImplementedError()

# load data
mode = 'train'
tokens_source = args.tokens_source
debug = True  # NOTE!!!!!!!!!!!!!!!!!!!!!!!!!WE ARE IN DEBUG MODE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

if tokens_source == 'origin':
    tests_ui_folder=PATH_TEST_UI
elif tokens_source == 'classifui':
    tests_ui_folder=PATH_PARSED_CLASSIFUI

dataset = UITestsDataset(tests_ui_folder, mode, debug)
print(f"UITestsDataset is initialized in mode {mode} from {tests_ui_folder} "
            f" with {tokens_source=}. {len(dataset)=}")


UITestsDataset is initialized in mode train from data/rust/tests/ui  with tokens_source='origin'. len(dataset)=233


In [68]:
# split data
seed = args.seed
traintestsplit = args.traintestsplit
lengths = [traintestsplit, 1-traintestsplit]

splitgenerator = Generator().manual_seed(seed)
train_set, val_set = random_split(dataset, 
    lengths=lengths, generator=splitgenerator)
print(f"Split generator initialized with {seed=}, {lengths=}.")

Split generator initialized with seed=42, lengths=[0.7, 0.30000000000000004].


In [81]:
concat_method = args.w2v_concat_method #  TODO (NotImplemented)
min_count = args.w2v_min_count
vector_size = args.w2v_vector_size
window = args.w2v_window
epochs = args.w2v_epochs

splitter, w2v_model = TrainW2VModel(train_set, tokens_source, min_count, vector_size, window, epochs).get_model()
collate_fn = W2VClassificationCollator(w2v_model, splitter, dataset.classes)

X_train, Y_train = collate_fn(train_set)
X_val,   Y_val   = collate_fn(val_set)

print(f"{len(dataset.classes)=}, {len(X_train)=}, {len(X_val)=}")

DataLoaders are initialized with batch_size=1
len(dataset.classes)=5, len(X_train)=164, len(X_val)=69


In [121]:
from sklearn.model_selection import train_test_split
X, Y = collate_fn(dataset)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size=0.7, shuffle=True, random_state=args.seed)

In [102]:
def fix_labels(Y_train, Y_val):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder().fit(Y_train)
    Y_train = le.fit_transform(Y_train)
    Y_val = le.fit_transform(Y_val)
    return Y_train, Y_val

In [122]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# gnb = GaussianNB()
gnb = BernoulliNB()
gnb.fit(X_train, Y_train)

BernoulliNB()

In [123]:
from sklearn.metrics import accuracy_score
Y_pred = gnb.predict(X_val)
print('Accuracy:', accuracy_score(Y_val, Y_pred))

Accuracy: 0.6571428571428571


In [124]:
Y_prob = gnb.predict_proba(X_val)

In [125]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
precision_recall_fscore_support(Y_val, Y_pred)

/home/huawei123/kwx1991442/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.        , 0.15384615, 0.33333333, 1.        , 1.        ]),
 array([0.        , 1.        , 1.        , 1.        , 0.61666667]),
 array([0.        , 0.26666667, 0.5       , 1.        , 0.7628866 ]),
 array([ 1,  4,  1,  4, 60]))

In [130]:
roc_auc_score(Y_val, Y_prob, multi_class='ovr')

0.9695375494071147

In [127]:
import numpy as np
np.unique(Y, return_counts=True) 

(array([0, 1, 2, 3, 4]), array([ 10,  15,   2,   8, 198]))

In [ ]:
# from gensim.models import doc2vec
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# #Train of the model Doc2Vec

# tag_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(training_docs)]


# model = doc2vec.Doc2Vec(vector_size = 300, window = 1, min_count = 3, workers = 6, epochs=20)

# model.build_vocab(tag_documents)
# model.train(tag_documents, total_examples=model.corpus_count, epochs=20)
# model.save("d2v.model")

In [101]:
np.unique(Y_val), Y_prob.shape

(array([0, 1, 3, 4]), (69, 5))